In [1]:
import io
import re
import json
import sqlite3
import subprocess
import tempfile
from pprint import pprint

import numpy as np
import pandas as pd
import requests as req

import matplotlib.pyplot as plt
import seaborn as sns

import ftfy
import mistune
import textstat

import textacy
import textacy.ke

from tqdm import tqdm

sns.set(context="talk", style="darkgrid")

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 999

In [2]:
conn = sqlite3.connect("imdb.db")

In [4]:
df = pd.read_sql("select * from tv", conn)
df.head()

,tv_id,name,start,end,genres,seasons,episodes,total_votes,average_votes,average_rating,trend,rank,poster,path
0,67251,Chernobyl,2019,2019,"Drama, History, Thriller",1,5,219554,43911,9.6,648,1,/assets/images/shows/chernobyl-poster-67251.jpg,/tv-shows/chernobyl-ratings-67251/
1,26649,Game of Thrones,2011,2019,"Action, Drama, Adventure",8,73,3484981,47739,8.8,"6,145",2,/assets/images/shows/game-of-thrones-poster-26...,/tv-shows/game-of-thrones-ratings-26649/
2,42161,Black Mirror,2011,-,"Drama, Sci-fi, Thriller",5,22,839953,38180,7.9,"2,991",3,/assets/images/shows/black-mirror-poster-42161...,/tv-shows/black-mirror-ratings-42161/
3,35707,Sherlock,2010,-,"Drama, Crime, Mystery",4,12,309700,25808,8.9,742,4,/assets/images/shows/sherlock-poster-35707.jpg,/tv-shows/sherlock-ratings-35707/
4,26165,Breaking Bad,2008,2013,"Drama, Crime, Thriller",5,62,1169385,18861,9.0,"6,979",5,/assets/images/shows/breaking-bad-poster-26165...,/tv-shows/breaking-bad-ratings-26165/


In [23]:
def download_episodes(row):
    tv_id = row.tv_id
    url = f"https://www.ratingraph.com/show-episodes-list/{tv_id}/?draw=2&columns%5B0%5D%5Bdata%5D=trend&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=false&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=season&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=false&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=episode&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=false&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=name&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=false&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=start&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=false&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=total_votes&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=false&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=average_rating&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=false&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=1&order%5B0%5D%5Bdir%5D=asc&order%5B1%5D%5Bcolumn%5D=2&order%5B1%5D%5Bdir%5D=asc&start=0&length=250&search%5Bvalue%5D=&search%5Bregex%5D=false&_=1588110434853 "
    data = req.get(url).json()

    episodes = pd.DataFrame(data["data"]).rename(columns={"name": "title"})
    episodes["tv_id"] = row.tv_id
    episodes["average_rating"] = episodes.average_rating.astype(float)
    episodes["total_votes"] = episodes.total_votes.str.replace(",", "").astype(int)
    episodes.to_sql("episode", conn, index=False, if_exists="append")

In [ ]:
for index, row in tqdm(df.head(8000).iterrows()):
    download_episodes(row)

465it [04:19,  2.07it/s]